In [34]:
import django_initializer
from backend.pymongo.mongodb import db
from datetime import datetime
import arrow
from api import models

In [56]:
__collection = db.api_order
def get_anonymous_buyers_data(user_subscription_id):
    query = [
        {
            "$match":{
                "user_subscription_id": user_subscription_id,
                "buyer_id":{'$eq':None}
            },
        },
        {
            "$group": {
                '_id':None,
                'order_id_list': {'$addToSet': '$id'},
            }
        }
    ]
    cursor = __collection.aggregate(query)
    return list(cursor)[0].get('order_id_list', [])

def get_registered_buyers_data(user_subscription_id):
    query = [
        {
            "$match":{
                "user_subscription_id": user_subscription_id,
                "buyer_id":{'$ne':None}
            }
        },
        {
            '$group': {
                '_id': '$buyer_id',
                'order_id': {'$first':'$id'}
                
            }
        },
        {
            "$project": {
                '_id':0,
                'order_id': '$order_id'
            }
        },
        {
            "$group": {
                '_id':None,
                'order_id_list': {'$addToSet': '$order_id'},
            }
        }
    ]
    cursor = __collection.aggregate(query)
    return list(cursor)[0].get('order_id_list', [])


In [57]:
anonymous_buyers_data = get_anonymous_buyers_data(218)
print(anonymous_buyers_data)
registered_buyers_data = get_registered_buyers_data(218)
print(registered_buyers_data)
buyers_id_data = anonymous_buyers_data + registered_buyers_data
print(buyers_id_data)
print(len(buyers_id_data))
queryset = models.order.order.Order.objects.filter(id__in=buyers_id_data)
print(len(queryset))

[33429, 32829, 33409, 33189, 33197]
[32855]
[33429, 32829, 33409, 33189, 33197, 32855]
6
6


In [58]:
queryset = models.order.order.Order.objects.filter(id__in=registered_buyers_data[0]['cart_id_list'])
print(queryset)

TypeError: 'int' object is not subscriptable